##### 1.1 Data Exploration
First, we explore the data such as the png images for all types(rock, paper and scissors), as well the csv files for train, test and devtest data. We use pandas for data exploration..


In [31]:
# Import the relevant libraries for Data Exploration and Visualization
import os
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
import cv2

In [32]:
# Define dataset paths
# Define the base directory
base_dir = r"Users\Admin\OneDrive\Desktop\AI for Text and Vison\assignment-2-marleybharat"
partitions = ["train", "devtest", "test"]
classes = ["rock", "paper", "scissors"]

In [33]:
# Task 1: 
# Display one image per class and print its dimensions
print("Displaying one image per class with dimensions:")

for class_name in classes:
    # Get the directory for the current class
    class_dir = os.path.join(base_dir, class_name)
    
    # Check if the directory exists and contains images
    print(f"Checking directory: {class_dir}")
    if not os.path.exists(class_dir):
        print(f"Directory not found: {class_dir}")
        continue
    
    # Filter valid image files
    image_files = [f for f in os.listdir(class_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
    if len(image_files) == 0:
        print(f"No valid image files found in {class_dir}")
        continue
    
    # Get the first image in the directory
    image_name = image_files[0]
    image_path = os.path.join(class_dir, image_name)
    
    # Load the image
    image = Image.open(image_path)
    image_array = np.array(image)
    
    # Display the image
    plt.imshow(image)
    plt.title(f"Class: {class_name}")
    plt.axis("off")
    plt.show()
    
    # Print the dimensions of the numpy array
    print(f"Image dimensions for class {class_name}: {image_array.shape}")

Displaying one image per class with dimensions:
Checking directory: Users\Admin\OneDrive\Desktop\AI for Text and Vison\assignment-2-marleybharat\rock
Directory not found: Users\Admin\OneDrive\Desktop\AI for Text and Vison\assignment-2-marleybharat\rock
Checking directory: Users\Admin\OneDrive\Desktop\AI for Text and Vison\assignment-2-marleybharat\paper
Directory not found: Users\Admin\OneDrive\Desktop\AI for Text and Vison\assignment-2-marleybharat\paper
Checking directory: Users\Admin\OneDrive\Desktop\AI for Text and Vison\assignment-2-marleybharat\scissors
Directory not found: Users\Admin\OneDrive\Desktop\AI for Text and Vison\assignment-2-marleybharat\scissors


In [28]:
# Task 2: Count the number of images per class
print("\nCounting the number of images per class:")

total_images = 0
for class_name in classes:
    # Get the directory for the current class
    class_dir = os.path.join(base_dir, class_name)
    
    # Check if the directory exists
    if not os.path.exists(class_dir):
        print(f"  {class_name}: Directory not found")
        continue
    
    # Filter valid image files
    image_files = [f for f in os.listdir(class_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
    num_images = len(image_files)
    total_images += num_images
    print(f"  {class_name}: {num_images} images")

# Print total images in the dataset
print(f"\nTotal images in the dataset: {total_images}")


Counting the number of images per class:
  rock: Directory not found
  paper: Directory not found
  scissors: Directory not found

Total images in the dataset: 0


##### 1.2 Data Preparation
At this stage, we prepare the data 

such as the png images for all types(rock, paper and scissors), as well the csv files for train, test and devtest data. 

We use pandas for data exploration..

Write code that generates the dataset and dataloaders that will be used for classification using PyTorch.

Make sure that the image data are re-scaled to values between 0 and 1, and that all images are re-sized to 100x100.



In [29]:
# Import the Pytorch libraries for model building
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
print("The device is", device)

The device is cpu


In [30]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Base directory for images (modify if needed)
base_dir = r"Users\Admin\OneDrive\Desktop\AI for Text and Vison\assignment-2-marleybharat"  # where the CSVs were uploaded

# Define the CSV files
csv_files = {
    "train": os.path.join(base_dir, "train.csv"),
    "devtest": os.path.join(base_dir, "devtest.csv"),
    "test": os.path.join(base_dir, "test.csv"),
}

# Image transformations: Resize + Scale to [0,1]
transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor(),  # Converts image to [0,1] and channel-first tensor
])

# Custom Dataset
class RockPaperScissorsDataset(Dataset):
    def __init__(self, csv_file, base_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.base_dir = base_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.base_dir, self.data.iloc[idx, 0])
        label = self.data.iloc[idx, 1]

        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

# Create datasets and dataloaders
batch_size = 32
dataloaders = {}

for split, csv_path in csv_files.items():
    dataset = RockPaperScissorsDataset(csv_path, base_dir, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    dataloaders[split] = dataloader

# ✅ Test the dataloaders
for split, loader in dataloaders.items():
    print(f"\nPartition: {split}")
    for images, labels in loader:
        print(f"  Image batch shape: {images.shape}")  # [batch_size, 3, 100, 100]
        print(f"  Label batch shape: {labels.shape}")
        print(f"  Labels sample: {labels[:5]}")
        break  # Just check first batch


FileNotFoundError: [Errno 2] No such file or directory: 'Users\\Admin\\OneDrive\\Desktop\\AI for Text and Vison\\assignment-2-marleybharat\\train.csv'